In [11]:
import torch
from torch import nn
import torch.optim as optim

import pickle
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader

In [12]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [13]:
# Загрузка и подготовка данных CIFAR-10
def get_cifar10_data(batch_size=64):  # Уменьшил batch_size для скорости
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)  # workers=0 для скорости

    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

    return trainloader, testloader

In [14]:
def build_alexnet(num_classes=10):
    def init_weights(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
        if type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    net = nn.Sequential(
        # Input: 32x32x3 (CIFAR-10)

        # Layer 1
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 16x16x64

        # Layer 2
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 8x8x128

        # Layer 3
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),  # Output: 8x8x256

        # Layer 4
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),  # Output: 8x8x256

        # Layer 5
        nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 4x4x128

        nn.Flatten(),

        # Fully Connected Layers
        nn.Dropout(0.5),
        nn.Linear(128 * 4 * 4, 512),
        nn.ReLU(inplace=True),

        nn.Dropout(0.5),
        nn.Linear(512, 256),
        nn.ReLU(inplace=True),

        nn.Linear(256, num_classes)
    )

    net.apply(init_weights)
    return net

In [15]:
def train_alexnet(net, train_loader, device, num_epochs=20, learning_rate=0.01):  # Уменьшено до 20 эпох
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
    loss_function = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)  # Более частый шаг
    acc_history = []

    with tqdm(total=len(train_loader)*num_epochs, position=0, leave=True) as pbar:
        for epoch in range(num_epochs):
            running_loss = 0.0
            correct = 0
            total = 0

            net.train()
            for batch_num, (inputs, labels) in enumerate(train_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                outputs = net(inputs)
                loss = loss_function(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                # Calculate batch Accuracy
                _, predicted = outputs.max(1)
                batch_total = labels.size(0)
                batch_correct = predicted.eq(labels).sum().item()
                batch_acc = batch_correct/batch_total

                pbar.set_description("Epoch: %d, Batch: %2d, Loss: %.4f, Acc: %.2f" %
                                   (epoch+1, batch_num+1, running_loss/(batch_num+1), batch_acc))
                pbar.update()

                total += batch_total
                correct += batch_correct

            scheduler.step()

            # Print the evaluation metric and reset it for the next epoch
            epoch_acc = correct/total
            acc_history.append(epoch_acc)
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, '
                  f'Train Acc: {epoch_acc:.2%}')

        pbar.close()

    return acc_history

In [16]:
# Функция оценки
def evaluate_acc(net, test_loader, device):
    net.eval()
    total = 0
    correct = 0

    with torch.no_grad():
        for batch_num, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = net(inputs)
            _, predicted = outputs.max(1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    acc = correct/total
    return acc

In [17]:
# Функция визуализации
def print_history(history, title):
    plt.figure(figsize=(7, 4))
    plt.plot(history)
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.grid(True)
    plt.show()

In [ ]:
batch_size = 64
epochs = 20
lr = 0.01

train_dataloader, test_dataloader = get_cifar10_data(batch_size)

print(f"Training samples: {len(train_dataloader.dataset)}")
print(f"Test samples: {len(test_dataloader.dataset)}")
print(f"Batch size: {batch_size}")
print(f"Epochs: {epochs}")

# Создание модели
alexnet = build_alexnet(num_classes=10)
alexnet = alexnet.to(device)
print("AlexNet architecture:")
print(alexnet)

# Обучение 
hist_alexnet = train_alexnet(alexnet, train_dataloader, device, epochs, lr)

# Оценка
alexnet_acc = evaluate_acc(alexnet, test_dataloader, device)
print(f'\nTest Accuracy (AlexNet): {alexnet_acc:.2%}')

# Визуализация
print_history(hist_alexnet, "AlexNet Model Accuracy")

Training samples: 50000
Test samples: 10000
Batch size: 64
Epochs: 20
AlexNet architecture:
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): Flatten(start_dim=1, end_dim=-1)
  (14): Dropout(p=0.5, inplace=False)
  (15): Linear(in_features=2048, out_features=512, bias=True)
  (16

Epoch: 1, Batch: 174, Loss: 2.1133, Acc: 0.25:   1%|          | 174/15640 [00:44<1:05:06,  3.96it/s]